In [2]:
import time
import requests
from time import sleep
import re, requests, csv
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium import webdriver
keys = Keys()

In [3]:
from selenium.webdriver.common.by import By

In [4]:
# 준비 동작
next_btn = []
for i in range(2, 12):
    next_btn.append(f'a:nth-child({i})')
next_btn.append("a.fAUKm1ewwo._2Ar8-aEUTq")

In [26]:
def get_links(keyword):
    move = int(input("몇 개의 상품을 검색하시겠습니까?"))
    shopAddress = "https://search.shopping.naver.com/search/all?query="
    driver = webdriver.Chrome()
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument("--hide-scrollbars")
    driver.get(f"{shopAddress} + {keyword}")
    print(f"{shopAddress}{keyword} 페이지로 이동했어요")
    hi = []
    isCompleted = False
    while(True):
        el = driver.find_elements(By.CLASS_NAME, "product_link__TrAac.linkAnchor")
        for j in el:
            href = j.get_attribute("href")
            if (href not in hi):
                hi.append(href)
                print(f"상품을 {len(hi)}개 모았습니다.")
                if (len(hi) == move):
                    isCompleted = True
                    break
        if (isCompleted): break
        webdriver.ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(1)
    driver.quit()

    return hi

In [135]:
def get_reviews_temp0(driver, stop):
    count = 0

    review_list = []

    el = driver.find_element(By.XPATH, '//*[@id="REVIEW"]/div/div[3]/div[1]/div[1]/ul/li[4]/a')
    actions = webdriver.ActionChains(driver).move_to_element(el).perform()
    actions2 = webdriver.ActionChains(driver).send_keys(Keys.ARROW_DOWN).perform()
    el.send_keys(Keys.ENTER)
    print("리뷰 낮은 순으로 정렬.")
    time.sleep(3)

    el = driver.find_element(By.CLASS_NAME, 'UlkDgu9gWI')
    webdriver.ActionChains(driver).move_to_element(el).perform()
    reviews_n = int(el.get_attribute("innerText").replace(",", ""))

    while (len(review_list) < stop and len(review_list) < reviews_n):
        # 리뷰 별 별점
        el = driver.find_element(By.CSS_SELECTOR, f"#REVIEW > div > div._180GG7_7yx > div.cv6id6JEkg > ul > li:nth-child({count % 20 + 1}) > div > div._30o7PGmsIy > div > div._1XNnRviOK8 > div > div._1YShY6EQ56 > div._1rZLm75kLm > div._37TlmH3OaI > div._2V6vMO_iLm > em")
        webdriver.ActionChains(driver).move_to_element(el).perform()
        star = int(el.get_attribute("innerText"))

        if star > 3:
            print("4점 이상의 리뷰는 수집하지 않습니다.")
            return review_list

        if (count % 20 == 0):
            el = driver.find_element(By.CSS_SELECTOR, "#REVIEW > div > div._180GG7_7yx > div.cv6id6JEkg > div > div > " + str(next_btn[(count // 20) % 11]) + '') # 리뷰 인덱스 페이지 이동
            actions = webdriver.ActionChains(driver).move_to_element(el)
            driver.implicitly_wait(3)
            actions.perform()
            el.click()
            time.sleep(2)
            html = driver.page_source
            soup = bs(html, "html.parser")
            reviews = soup.find_all('div', class_='YEtwtZFLDz')
            print(f"불러온 리뷰 : {len(reviews)}개")
        review = reviews[count % 20].text
        review = re.sub('[^#0-9a-zA-Zㄱ-ㅣ가-힣 ]', "", review)
        if ("더보기" in review):
            review = soup.find_all('div', class_="YEtwtZFLDz.vlXMQEAtPR")
            review = review[count % 20].text
            review = re.sub('[^#0-9a-zA-Zㄱ-ㅣ가-힣 ]', "", review)
        review_list.append(review)
        count = count + 1
        print(f'{count}번째 리뷰 가져옴.')
    driver.quit()

    return review_list

In [109]:
def get_reviews_temp1(driver, stop):
    review_list = []

    for i in range(5):
        reviews_temp = get_reviews_temp1_score(driver, stop // 5, i, 3)
        for review in reviews_temp:
            review_list.append(review)

    driver.quit()

    return review_list

def get_reviews_temp1_score(driver, stop_score, order, max_score):
    review_list_score = []

    count = 0
    score = 0

    try:
        el = driver.find_element(By.XPATH, f'//*[@id="section_review"]/div[2]/div[2]/ul/li[{order + 2}]/a')
        webdriver.ActionChains(driver).move_to_element(el).click().perform()
    except:
        print(f"{order}순서 리뷰가 없습니다.")
        return []
    
    time.sleep(2)
    
    score = int(el.get_attribute("innerText").split("점")[0])
    reviews_n = int(el.get_attribute("innerText").split("(")[1].split(")")[0])

    if score > max_score:
        print(f"{score}점 리뷰는 건너뜁니다.")
        return []
    
    print(f"{score}점 리뷰를 가져왔습니다.")

    time.sleep(3)

    while (len(review_list_score) < stop_score and len(review_list_score) < reviews_n):
        if (count % 20 == 0):
            if (count != 0):
                el = driver.find_element(By.CSS_SELECTOR, "#section_review > div.pagination_pagination__JW7zT > " + str(next_btn[(count // 20) % 11]) + '')
                webdriver.ActionChains(driver).move_to_element(el).click().perform()
                time.sleep(2)
            html = driver.page_source
            soup = bs(html, "html.parser")
            reviews = soup.find_all("p", class_="reviewItems_text__XrSSf")
            print(f"불러온 리뷰 : {len(reviews)}개")
        review = reviews[count % 20].text
        review = re.sub('[^#0-9a-zA-Zㄱ-ㅣ가-힣 ]', "", review)
        review_list_score.append(review)
        count = count + 1
        print(f'{count}번째 리뷰 가져옴.')

    return review_list_score

In [131]:
def get_reviews(address, stop):
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument("--hide-scrollbars")
    chromeOptions.add_argument("headless")
    chromeOptions.add_argument("disable-gpu")
    chromeOptions.add_argument("disable-infobars")
    chromeOptions.add_argument("--disable-extensions")

    prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 'plugins' : 2, 'popups': 2, 'geolocation': 2, 'notifications' : 2, 'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 'mixed_script': 2, 'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2, 'protocol_handlers' : 2, 'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2, 'durable_storage' : 2}}   
    chromeOptions.add_experimental_option('prefs', prefs)

    driver = webdriver.Chrome(options=chromeOptions)
    driver.maximize_window()

    driver.get(address)
    page = requests.get(address)

    form = -1

    # stop이 총 리뷰 수보다 많다면, 총 리뷰 수로 계산하기

    print(f"{address} 페이지 리뷰 크롤링을 시작합니다.")

    review_list = []

    driver.implicitly_wait(3)
    
    # 리뷰 보기 버튼 클릭
    try:
        print("페이지의 형태가 0인지 검사하고 있습니다.")
        el = driver.find_element(By.CSS_SELECTOR, '#content > div > div.z7cS6-TO7X > div._27jmWaPaKy > ul > li:nth-child(2) > a')
        webdriver.ActionChains(driver).move_to_element(el).click().perform()
        print("리뷰 창으로 넘어감")
        form = 0
    except:
        try:
            print("페이지의 형태가 1인지 검사하고 있습니다.")
            el = driver.find_element(By.CSS_SELECTOR, "#snb > ul > li:nth-child(2) > a")
            webdriver.ActionChains(driver).move_to_element(el).click().perform()
            print("리뷰 창으로 넘어감")
            form = 1
        except:
            print("페이지 형태를 찾지 못했습니다.")
            return []

    time.sleep(10)

    # 페이지 형태 별로 리뷰 긁어오는 방식이 다름.
    if form == 0:
        print("페이지의 형태가 0입니다.")
        review_list = get_reviews_temp0(driver, stop)
    elif form == 1:
        print("페이지의 형태가 1입니다.")
        review_list = get_reviews_temp1(driver, stop)
    else:
        print("처음 보는 형태의 페이지예요.")
        return []
    
    return review_list

In [74]:
PAGES = get_links("기내용 캐리어")

https://search.shopping.naver.com/search/all?query=기내용 캐리어 페이지로 이동했어요
상품을 1개 모았습니다.
상품을 2개 모았습니다.
상품을 3개 모았습니다.
상품을 4개 모았습니다.


In [136]:
review_pages = {}
for i in PAGES:
    review_pages[i] = get_reviews(i, 30)

https://cr.shopping.naver.com/adcr.nhn?x=Jw0F1pH%2FjxM0b92Kt562fP%2F%2F%2Fw%3D%3DsYZ1o0eGEbLz6JUCg01nqFN5AnigGJS7uv%2BloWTKhT7Ey2kyWyP9m4E1XduFzvzi2KCR4Ol6VzE%2FHPbyEa1b0EglEKcCvll57Fv12yQC3j%2FWQDmLdNZE%2Bn3SQpNGiVpr%2BTHJmy0hO%2F44rzyIho3OIevQSwZImSpdb5fnHNZoisjK84OS6LYm2VrPehWjlu%2Fp3jj9KGBW69ZED30eD2DXGkBmJF8FPdBKU1BQQRrWiSHVMwVDBi6sv3j9lf43hXE7nVV3pn6Znl%2B9MAtsEE%2BZ8ouacYsA2T1ONC2M4R1%2F3Eb%2Fl0JEFNxXGbEg3dzcS%2B9YOnx0JBi7qYF9MraMxoiIbrSvZN8an914mTNs9WWbrXBCwh3RHqg9np85PbUkZZA0bBlSY0DiDEtN5td2AJ8xQ0DJhIixXejY%2BXOu4FHR9CtuiavlapONVIbIU1RUOXa%2F7RmHAiTrZaidwTqbZOc2glI54sw8ziJU0KnM1TAYrPU%2B3v4%2FW%2FS1mPt6DTaD3hW%2BFrJ0zrJrImpCmhe1%2Fv3qMz3UNqLqN4y49MOfMF6hSdBILAjoEvAEC8Gn1MUZeDPjLIBtAavaz5K7Os2kFf6WfKiQMdZDio4uWDvt%2FdlQZsiMDfkLIyuldBS3f8m8e6nLmzUhNzs2VE%2B%2Bisqs3JAEppg%3D%3D&nvMid=85089354016&catId=50007808 페이지 리뷰 크롤링을 시작합니다.
페이지의 형태가 0인지 검사하고 있습니다.
리뷰 창으로 넘어감
페이지의 형태가 0입니다.
리뷰 낮은 순으로 정렬.
불러온 리뷰 : 20개
1번째 리뷰 가져옴.
2번째 리뷰 가져옴.
3번째 리뷰 가져옴.
4번째 리뷰 가져옴.
5번째 리뷰 가져옴.
6번째 리뷰 가져옴.
7번째 

In [76]:
PAGES

['https://cr.shopping.naver.com/adcr.nhn?x=Jw0F1pH%2FjxM0b92Kt562fP%2F%2F%2Fw%3D%3DsYZ1o0eGEbLz6JUCg01nqFN5AnigGJS7uv%2BloWTKhT7Ey2kyWyP9m4E1XduFzvzi2KCR4Ol6VzE%2FHPbyEa1b0EglEKcCvll57Fv12yQC3j%2FWQDmLdNZE%2Bn3SQpNGiVpr%2BTHJmy0hO%2F44rzyIho3OIevQSwZImSpdb5fnHNZoisjK84OS6LYm2VrPehWjlu%2Fp3jj9KGBW69ZED30eD2DXGkBmJF8FPdBKU1BQQRrWiSHVMwVDBi6sv3j9lf43hXE7nVV3pn6Znl%2B9MAtsEE%2BZ8ouacYsA2T1ONC2M4R1%2F3Eb%2Fl0JEFNxXGbEg3dzcS%2B9YOnx0JBi7qYF9MraMxoiIbrSvZN8an914mTNs9WWbrXBCwh3RHqg9np85PbUkZZA0bBlSY0DiDEtN5td2AJ8xQ0DJhIixXejY%2BXOu4FHR9CtuiavlapONVIbIU1RUOXa%2F7RmHAiTrZaidwTqbZOc2glI54sw8ziJU0KnM1TAYrPU%2B3v4%2FW%2FS1mPt6DTaD3hW%2BFrJ0zrJrImpCmhe1%2Fv3qMz3UNqLqN4y49MOfMF6hSdBILAjoEvAEC8Gn1MUZeDPjLIBtAavaz5K7Os2kFf6WfKiQMdZDio4uWDvt%2FdlQZsiMDfkLIyuldBS3f8m8e6nLmzUhNzs2VE%2B%2Bisqs3JAEppg%3D%3D&nvMid=85089354016&catId=50007808',
 'https://cr.shopping.naver.com/adcr.nhn?x=oqg226ETIA2Hs48HqtE6nP%2F%2F%2Fw%3D%3DsdrG4pyBfhvMXw%2F3oii4MhYXPJP5wVY8nB%2FlTKb1X6Xsy2kyWyP9m4E1XduFzvzi2KCR4Ol6VzE%2FHPbyE

In [137]:
for i in PAGES:
    print(f"다음 페이지의 리뷰를 보여드릴게요 : {i}")
    for j in review_pages[i]:
        print(j)

다음 페이지의 리뷰를 보여드릴게요 : https://cr.shopping.naver.com/adcr.nhn?x=Jw0F1pH%2FjxM0b92Kt562fP%2F%2F%2Fw%3D%3DsYZ1o0eGEbLz6JUCg01nqFN5AnigGJS7uv%2BloWTKhT7Ey2kyWyP9m4E1XduFzvzi2KCR4Ol6VzE%2FHPbyEa1b0EglEKcCvll57Fv12yQC3j%2FWQDmLdNZE%2Bn3SQpNGiVpr%2BTHJmy0hO%2F44rzyIho3OIevQSwZImSpdb5fnHNZoisjK84OS6LYm2VrPehWjlu%2Fp3jj9KGBW69ZED30eD2DXGkBmJF8FPdBKU1BQQRrWiSHVMwVDBi6sv3j9lf43hXE7nVV3pn6Znl%2B9MAtsEE%2BZ8ouacYsA2T1ONC2M4R1%2F3Eb%2Fl0JEFNxXGbEg3dzcS%2B9YOnx0JBi7qYF9MraMxoiIbrSvZN8an914mTNs9WWbrXBCwh3RHqg9np85PbUkZZA0bBlSY0DiDEtN5td2AJ8xQ0DJhIixXejY%2BXOu4FHR9CtuiavlapONVIbIU1RUOXa%2F7RmHAiTrZaidwTqbZOc2glI54sw8ziJU0KnM1TAYrPU%2B3v4%2FW%2FS1mPt6DTaD3hW%2BFrJ0zrJrImpCmhe1%2Fv3qMz3UNqLqN4y49MOfMF6hSdBILAjoEvAEC8Gn1MUZeDPjLIBtAavaz5K7Os2kFf6WfKiQMdZDio4uWDvt%2FdlQZsiMDfkLIyuldBS3f8m8e6nLmzUhNzs2VE%2B%2Bisqs3JAEppg%3D%3D&nvMid=85089354016&catId=50007808
솔직후기를 남기겠습니다후기가 좋아서 가격이 저렴해서 구매를 했지만에이블리에 더 저렴한 3만원짜리 캐리어도 있고 차라리 돈을 더 주고 다른 캐리어를 구매할 거 같네요3박 4일 제주도 여행을 갔다온 다음 캐리어 상태입니다 스크래치에 너무 취약하고요 너무 심해서 물티슈로 한번